## 딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회

**※주의** : 반드시 본 파일을 이용하여 제출을 수행해야 하며, 파일의 이름은 `task.ipynb`로 유지되어야 합니다.

* #### 추론 실행 환경
    * `python 3.9` 환경
    * `CUDA 10.2`, `CUDA 11.8`, `CUDA 12.6`를 지원합니다.
    * 각 CUDA 환경에 미리 설치돼있는 torch 버전은 다음 표를 참고하세요.

<table>
  <thead>
    <tr>
      <th align="center">Python</th>
      <th align="center">CUDA</th>
      <th align="center">torch</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center" style="vertical-align: middle;">3.8</td>
      <td align="center">10.2</td>
      <td align="center">1.6.0</td>
    </tr>
    <tr>
      <td align="center" style="vertical-align: middle;">3.9</td>
      <td align="center">11.8</td>
      <td align="center">1.8.0</td>
    </tr>
    <tr>
      <td align="center">3.10</td>
      <td align="center">12.6</td>
      <td align="center">2.7.1</td>
    </tr>
  </tbody>
</table>

#### CUDA 버전 관련 안내사항
  - 이번 경진대회는 3개의 CUDA 버전을 지원합니다.  
  - 참가자는 자신의 모델의 라이브러리 의존성에 맞는 CUDA 환경을 선택하여 모델을 제출하면 됩니다.   
  - 각 CUDA 환경에는 기본적으로 torch가 설치되어 있으나, 참가자는 제출하는 CUDA 버전과 호환되는 torch, 필요한 버전의 라이브러리를 `!pip install` 하여 사용하여도 무관합니다.

* #### `task.ipynb` 작성 규칙
코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하세요.   
1. **제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치**
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 그 외로, 모델 추론에 필요한 라이브러리를 직접 설치합니다.
2. **추론용 코드 작성**
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. **aif.submit() 함수를 호출하여 최종 결과를 제출**
    - **마이 페이지-활동히스토리**에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.
    - **※주의** : 제출하고자 하는 CUDA 환경에 맞는 key를 입력하여야 합니다.

<table>
  <thead>
    <tr>
      <th align="left">Competition 이름</th>
      <th align="center">CUDA</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회</td>
      <td align="center">11.8</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 12.6</td>
      <td align="center">12.6</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 10.2</td>
      <td align="center">10.2</td>
    </tr>
  </tbody>
</table>

------

#### 1. 제출용 aifactory 라이브러리 설치
※ 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [ ]:
!pip install -U aifactory

* 자신의 모델 추론 실행에 필요한 추가 라이브러리 설치

In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install numpy==1.23.5
!pip install scipy==1.11.4
!pip install scikit-learn==1.3.2
!pip install opencv-python-headless==4.9.0.80
!pip install pandas==2.0.3
!pip install Pillow

!pip install insightface
!pip install onnxruntime-gpu

!pip install efficientnet-pytorch==0.7.1
!pip install timm==0.9.12

!pip install scikit-image==0.21.0
!pip install tqdm==4.65.0
!pip install pyyaml==6.0

-----

#### 2. 추론용 코드 작성

##### 추론 환경의 기본 경로 구조

- 평가 데이터셋 경로: `./data/`
   - 채점에 사용될 테스트 데이터셋은 `./data/` 디렉토리 안에 포함되어 있습니다.
   - 해당 디렉토리에는 이미지(JPG, PNG)와 동영상(MP4) 파일이 별도의 하위 폴더 없이 혼합되어 있습니다.
```bash
/aif/
└── data/
    ├── {이미지 데이터1}.jpg
    ├── {이미지 데이터2}.png
    ├── {동영상 데이터1}.mp4
    ├── {이미지 데이터3}.png
    ├── {동영상 데이터2}.mp4
    ...
```

- 모델 및 자원 경로: 예시 : `./model/`
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치해야하 하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)

* 제출 파일은 `submission.csv`로 저장돼야 합니다.
  * submission.csv는 *filename*과 *label* 컬럼으로 구성돼야 합니다.
  * filename은 추론한 파일의 이름(확장자 포함), label은 추론 결과입니다. (real:0, fake:1)
  * filename은 *string*, label은 *int* 자료형이어야 합니다.
  * 추론하는 데이터의 순서는 무작위로 섞여도 상관 없습니다.

<table>
  <thead>
    <tr>
      <th align="center">filename</th>
      <th align="center">label</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center">{이미지 데이터1}.jpg</td>
      <td align="center">0</td>
    </tr>
    <tr>
      <td align="center">{동영상 데이터1}.mp4</td>
      <td align="center">1</td>
    </tr>
    <tr>
      <td colspan="2" align="center">...</td>
    </tr>
  </tbody>
</table>

**※ 주의 사항**

* argparse 사용시 `args, _ = parser.parse_known_args()`로 인자를 지정하세요.   
   - `args = parser.parse_args()`는 jupyter에서 오류가 발생합니다.
* return 할 결과물과 양식에 유의하세요.

In [ ]:
import os
import sys
import yaml
from pathlib import Path
# insightface가 모델을 찾을 경로를 'insightface_cache' 폴더로 강제(네트워크 차단 우회)
MODEL_CACHE_DIR = Path("./insightface_cache")
os.environ['INSIGHTFACE_HOME'] = str(MODEL_CACHE_DIR.resolve())
local_model_root = os.environ.get('INSIGHTFACE_HOME')

# 모델이 제출 폴더에 없으면 추론을 중단하고 오류 발생
# buffalo_l 모델의 필수 파일이 있는지 확인
if not (MODEL_CACHE_DIR / "models/buffalo_l").exists():
    raise FileNotFoundError(f"InsightFace model 'buffalo_l' not found at {MODEL_CACHE_DIR / 'models/buffalo_l'}. Please check your submission contents.")

print(f"Set INSIGHTFACE_HOME to: {local_model_root}")
print("✅ Local InsightFace model cache verified.")

import cv2
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from typing import Optional, Tuple, List, Dict

# --- 1. 프로젝트 모듈 임포트 ---
# task.ipynb와 같은 위치에 src 폴더 배치
sys.path.insert(0, ".")
# 모듈 임포트
from src.feature_extraction.feature_extractor import DeepfakeDetectionModel
from src.preprocessing.face_detector import create_face_detector
from src.preprocessing.face_aligner import FaceAligner

print("✅ 프로젝트 모듈 임포트 완료")

# --- 2. 상수 및 경로 설정 ---
# 학습된 모델 가중치 경로
MODEL_PATH = "./model/best_model.pth"
# 설정 파일 경로
PREPROCESSING_CONFIG = "./config/preprocessing_config.yaml"
TRAINING_CONFIG = "./config/model_config.yaml"

# 대회 환경 경로
DATA_PATH = Path("./data")
OUTPUT_CSV = "submission.csv"

# 기타 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
VIDEO_EXTS = {".mp4"}
FRAME_SAMPLE_COUNT = 5 # 비디오당 샘플링할 프레임 수

# --- 3. 설정 파일 로드 및 환경 설정 ---
print(f"Running on device: {DEVICE}")

with open(PREPROCESSING_CONFIG, 'r', encoding='utf-8') as f:
    pre_config = yaml.safe_load(f)
with open(TRAINING_CONFIG, 'r', encoding='utf-8') as f:
    train_config = yaml.safe_load(f)

# 추론 환경에 맞게 설정 강제
pre_config['detection']['device'] = DEVICE.type
train_config['hardware']['device'] = DEVICE.type

print("✅ 설정 파일 로드 완료")

# --- 4. 모델 및 전처리 모듈 로드 ---

# 1) 전처리 모듈 (FaceDetector, FaceAligner)
detector = create_face_detector(pre_config['detection'])
aligner = FaceAligner(pre_config['alignment'])

# 2) 딥페이크 탐지 모델 로드
model_cfg = train_config['model']
model = DeepfakeDetectionModel(
    num_classes=model_cfg['classifier']['num_classes'],
    pretrained=False, # 학습된 가중치를 로드하므로 False
    feature_extractor_config=model_cfg['feature_extractor'],
    classifier_hidden_dims=model_cfg['classifier']['hidden_dims'],
    dropout_rate=model_cfg['classifier']['dropout_rate']
)

# 학습된 가중치(checkpoint) 로드
if not Path(MODEL_PATH).exists():
    print(f"❌ 에러: 모델 가중치 파일을 찾을 수 없습니다! {MODEL_PATH}")
else:
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(DEVICE)
    model.eval() # 추론 모드로 설정
    print(f"✅ 모델 로드 및 추론 모드 설정 완료: {MODEL_PATH}")

# 3) 이미지 정규화 텐서
MEAN_TENSOR = torch.tensor([0.485, 0.456, 0.406], device=DEVICE).view(3, 1, 1)
STD_TENSOR = torch.tensor([0.229, 0.224, 0.225], device=DEVICE).view(3, 1, 1)

# --- 5. 핵심 추론 함수 정의 ---

def preprocess_frame(frame_bgr: np.ndarray) -> Optional[Tuple[torch.Tensor, torch.Tensor]]:
    """
    단일 프레임(BGR)을 받아 전처리하고 모델 입력 텐서(이미지, 랜드마크)를 반환합니다.
    """
    try:
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        detection = detector.detect(frame_rgb)
        if detection is None:
            return None

        aligned_face, tform_matrix = aligner.align(frame_rgb, detection['landmarks'])
        aligned_landmarks = aligner.get_aligned_landmarks(
            detection['landmarks'],
            tform_matrix
        )

        img_tensor = torch.from_numpy(aligned_face.transpose(2, 0, 1)).float().to(DEVICE)
        img_tensor = (img_tensor / 255.0 - MEAN_TENSOR) / STD_TENSOR

        lm_tensor = torch.from_numpy(aligned_landmarks).float().to(DEVICE)

        return img_tensor, lm_tensor
    except Exception as e:
        return None

def predict_single_file(file_path: Path) -> int:
    """
    단일 파일(이미지 또는 비디오) 경로를 받아 딥페이크 여부(0 또는 1)를 반환합니다.
    """
    frame_tensors = []
    landmark_tensors = []
    ext = file_path.suffix.lower()

    try:
        if ext in IMAGE_EXTS:
            frame = cv2.imread(str(file_path))
            if frame is not None:
                result = preprocess_frame(frame)
                if result:
                    frame_tensors.append(result[0])
                    landmark_tensors.append(result[1])

        elif ext in VIDEO_EXTS:
            cap = cv2.VideoCapture(str(file_path))
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            if total_frames > 0:
                frame_indices = np.linspace(0, total_frames - 1, FRAME_SAMPLE_COUNT, dtype=int)

                for idx in frame_indices:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                    ret, frame = cap.read()
                    if not ret or frame is None:
                        continue

                    result = preprocess_frame(frame)
                    if result:
                        frame_tensors.append(result[0])
                        landmark_tensors.append(result[1])
            cap.release()

        # 3. 예측 집계 (Aggregation)
        if not frame_tensors:
            return 0 # 얼굴 검출 실패 시 Real(0)

        images_batch = torch.stack(frame_tensors)
        landmarks_batch = torch.stack(landmark_tensors)

        with torch.no_grad():
            logits, _ = model(images_batch, landmarks_batch)
            probs = F.softmax(logits, dim=1) # (N, 2)

        avg_fake_prob = probs[:, 1].mean().item()
        final_label = 1 if avg_fake_prob >= 0.5 else 0
        return final_label

    except Exception as e:
        print(f"🔥 파일 처리 오류 {file_path.name}: {e}")
        return 0 # 예측 실패 시 Real(0)

# --- 6. 메인 실행 로직 ---
print("🚀 추론을 시작합니다...")

all_files = [p for p in DATA_PATH.iterdir() if p.is_file() and p.suffix.lower() in (IMAGE_EXTS | VIDEO_EXTS)]
results = []

for file_path in tqdm(all_files, desc="테스트 데이터 추론 중"):
    label = predict_single_file(file_path)
    results.append({
        "filename": file_path.name,
        "label": label
    })

# --- 7. submission.csv 파일 생성 ---
if results:
    submission_df = pd.DataFrame(results)
    submission_df['label'] = submission_df['label'].astype(int)
    submission_df = submission_df[["filename", "label"]]
    submission_df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ 추론 완료! 결과가 {OUTPUT_CSV} 파일로 저장되었습니다.")
else:
    # 빈 submission.csv라도 생성
    pd.DataFrame(columns=["filename", "label"]).to_csv(OUTPUT_CSV, index=False)
    print("⚠️ 처리할 파일을 찾지 못했으나, 빈 submission.csv를 생성합니다.")

print("모든 추론 과정이 종료되었습니다.")

----

#### 3. `aif.submit()` 함수를 호출하여 최종 결과를 제출

**※주의** : task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9197/baseline) 탭에 기재된 가이드라인을 따라 task 별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 task에 맞는 자신의 key를 사용해야 합니다.
-  **NOTE** : 이번 경진대회에서는 3개의 CUDA 버전을 지원하며, 각 CUDA 버전에 따라 task key가 상이합니다. 함수를 실행하기 전에 현재 key가 제출하고자 하는 CUDA 환경에 대한 key인지 반드시 확인하세요.

In [ ]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="faker",
    key="eb78b4fd-975e-496c-9dbe-acbc76123fb2"
)
#-----------------------------------------------------#
print(time.time() - t)